In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
#geo_home_france=pd.read_csv(header=0,index_col=0,filepath_or_buffer="/datastore/complexnet/jlevyabi/ml_soc_econ/data_files/UKSOC_rep/dgeo_france_most_freq.csv")
#geo_home_france.head()

In [3]:
import pickle 
d=pickle.load(open('d_mob_study_out.p','rb'))

In [4]:
import pickle
new_dic_real_france=d['new_dic_real_france']
new_dic_real_home=d['new_dic_real_home']

pre_filter_dic_income_moded_france=d['pre_filter_dic_income_moded_france']
pre_filter_dic_income_moded_home=d['pre_filter_dic_income_moded_home']
pre_filter_dic_income_weighted_france=d['pre_filter_dic_income_weighted_france']
pre_filter_dic_income_weighted_home=d['pre_filter_dic_income_weighted_home']

dic_income_moded_france=d['dic_income_moded_france']
dic_income_moded_home=d['dic_income_moded_home']
dic_income_weighted_france=d['dic_income_weighted_france']
dic_income_weighted_home=d['dic_income_weighted_home']

In [5]:
def get_pandas_from_dic(dic,weight=1,prec=3):
    if weight:
        return pd.DataFrame([(k,round(v[0],prec),v[1],v[2][0],v[2][1],v[2][2]) for k,v in tqdm(dic.items()) if v[2]!=None],
                           columns=["usr","income","std_income","avg_dist","max_dist","nb_locs"])
    else:
        return pd.DataFrame([(k,v[1],v[0][0],v[0][1],v[2]) for k,v in tqdm(dic.items()) ],
                           columns=["usr","income","lat","lon","rate_of_appear"])

In [8]:
pre_filter_df_income_moded_france=get_pandas_from_dic(pre_filter_dic_income_moded_france,weight=0)
pre_filter_df_income_moded_home=get_pandas_from_dic(pre_filter_dic_income_moded_home,weight=0)
pre_filter_df_income_weighted_france=get_pandas_from_dic(pre_filter_dic_income_weighted_france)
pre_filter_df_income_weighted_home=get_pandas_from_dic(pre_filter_dic_income_weighted_home)

df_income_moded_france=get_pandas_from_dic(dic_income_moded_france,weight=0)
df_income_moded_home=get_pandas_from_dic(dic_income_moded_home,weight=0)
df_income_weighted_france=get_pandas_from_dic(dic_income_weighted_france)
df_income_weighted_home=get_pandas_from_dic(dic_income_weighted_home)


100%|██████████| 14104/14104 [00:00<00:00, 38818.64it/s]

(9721, 6)

In [24]:
import numpy as np
tweets_df=pd.read_csv(open('/datastore/complexnet/jlevyabi/ml_soc_econ/data_files/UKSOC_rep/tweets/individual_files/all_together.csv','rU'),header=None, encoding='utf-8',sep=';',engine='c')
tweets_df.columns = ['usr', 'body']

/home/jlevyabi/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:2: DeprecationWarning: 'U' mode is deprecated
  from ipykernel import kernelapp as app


In [27]:
df_income_weighted_home['usr']=df_income_weighted_home['usr'].apply(lambda x: str(x))
#geo_home_france['usr']=geo_home_france['usr'].apply(lambda x: str(x))

In [112]:
df_income_weighted_home.head()

,usr,income,std_income,avg_dist,max_dist,nb_locs
0,1263741576,20055.510,0.000000,2.170473,12.053558,53
1,123306709,14117.010,0.000000,0.110266,1.090481,23
2,800994612,24690.711,0.000000,0.420870,26.373603,50
3,357570731,16932.391,1025.579742,0.618389,9.505615,52
4,459667002,19354.318,0.000000,0.003495,0.010418,89


In [161]:
tryo=(tweets_df.dropna(how='any').groupby('usr',squeeze=True,)['body'].apply(lambda x: "%s" % ', '.join(x))).to_frame()
tryo['usr'] = tryo.index
user_info=pd.merge(tryo, df_income_weighted_home, on='usr')
user_info.head()

,body,usr,income,std_income,avg_dist,max_dist,nb_locs
0,me suis fait virer d'un leader price comme une...,1002650276,21319.826,584.178432,4.940912,26.066781,33
1,"flemme fatale 😴, 🛩, dans une heure vacances ...",1003788668,20524.750,0.000000,0.737638,5.139245,13
2,"ah mince, gare de livron. bâtiment désaffect...",100460784,17151.398,2922.129791,0.076255,0.147308,4
3,"j-1 ✈️🇪🇸, j-6 👙🌴🕶☀️🌊🍹🏖🔥💦😍, départ pour rome ...",1006013748,21040.156,0.000000,1.221213,6.876153,10
4,"il faut arrêter de retourner la situation, ins...",1006462345,15681.566,0.000000,0.001411,0.008453,6


In [57]:
import gensim.models.word2vec as w2v
from gensim.models import Word2Vec
from gensim.similarities import MatrixSimilarity
from gensim.matutils import Dense2Corpus
from scipy import sparse
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import gensim
from tqdm import tqdm
model = Word2Vec.load('/home/jlevyabi/seacabo/data_files/lowe_dim_sosweet2vec.w2v')
def after_treatment_word2vec(w2v_model,min_times):
    treated_words=[]
    for word in w2v_model.wv.vocab:
        if w2v_model.wv.vocab[word].count>min_times:
            treated_words.append(word)
    return treated_words
vocab = after_treatment_word2vec(model,min_times=100)
print(len(vocab))

66604


In [162]:
#filter_usr_info=user_info[(user_info.total_geo>5)  ]
#print(filter_usr_info.shape)
import pickle
#phrunch_clusters=pickle.load(open('/home/jlevyabi/seacabo/data_files/spec_clusters.p','rb'))
with open('/home/jlevyabi/seacabo/data_files/spec_clusters.p', 'rb') as f:
    u = pickle._Unpickler(f)
    u.encoding = 'utf-8'
    phrunch_clusters = u.load()

word2cluster={word:cluster_nb for cluster_nb,cluster_words in phrunch_clusters.items() for word in cluster_words}
len(word2cluster)

42883

In [167]:
user_info.head()

,body,usr,income,std_income,avg_dist,max_dist,nb_locs,cfd_v1
0,me suis fait virer d'un leader price comme une...,1002650276,21319.826,584.178432,4.940912,26.066781,33,"[0.00376736519896, 0.00488580174241, 0.0015893..."
1,"flemme fatale 😴, 🛩, dans une heure vacances ...",1003788668,20524.750,0.000000,0.737638,5.139245,13,"[0.00681302043906, 0.00681302043906, 0.0037850..."
2,"ah mince, gare de livron. bâtiment désaffect...",100460784,17151.398,2922.129791,0.076255,0.147308,4,"[0.00421163464069, 0.00388260068439, 0.0011845..."
3,"j-1 ✈️🇪🇸, j-6 👙🌴🕶☀️🌊🍹🏖🔥💦😍, départ pour rome ...",1006013748,21040.156,0.000000,1.221213,6.876153,10,"[0.00417962674961, 0.00106920684292, 9.7200622..."
4,"il faut arrêter de retourner la situation, ins...",1006462345,15681.566,0.000000,0.001411,0.008453,6,"[0.00760240553727, 0.00238284352661, 0.0007942..."


In [163]:
from collections import Counter
from tqdm import tqdm
clust_freq=[]
for tw in tqdm(user_info.body):
    try:
        clust_freq.append((Counter([word2cluster[word] for word in tw.split() if word in word2cluster ])))
    except:
        clust_freq.append({1:None})
        continue


100%|██████████| 1304/1304 [00:30<00:00, 42.21it/s]

In [166]:
pre_cfd_v1=[{k:(v+0.0)/(sum(dic_count.values()))for k,v in dic_count.items() if v!=None} for dic_count in tqdm(clust_freq)]
user_info["cfd_v1"]=[np.array(list({clus:(dic_count[clus] if clus in dic_count else 0) 
                           for clus in range(len(phrunch_clusters))}.values())) for dic_count in tqdm(pre_cfd_v1)]


100%|██████████| 1304/1304 [00:00<00:00, 9158.16it/s]

In [168]:
feats=np.vstack((user_info.cfd_v1).values)
target=user_info.income.values

sorted_target=np.sort(target)
nb_class=3
target_bins=[sorted_target[i*int(len(sorted_target)/nb_class)] for i in range(nb_class)]
target_bins.append(max(sorted_target)+1)
target_class=np.digitize(target,target_bins)

In [204]:
3

array([2, 1, 1, 3, 1, 3, 2, 1, 2, 3, 3, 1, 1, 3, 2, 1, 1, 2, 1, 3, 1, 2, 2,
       1, 2, 1, 1, 1, 3, 1, 2, 2, 1, 1, 1, 1, 3, 2, 1, 1, 1, 1, 2, 1, 1, 1,
       1, 3, 1, 1, 2, 2, 1, 1, 3, 1, 3, 2, 2, 2, 1, 1, 1, 1, 3, 1, 3, 2, 1,
       1, 3, 3, 2, 3, 3, 1, 3, 3, 3, 1, 3, 1, 2, 2, 3, 2, 1, 3, 3, 1, 2, 1,
       3, 1, 3, 1, 1, 1, 2, 2, 2, 3, 3, 1, 1, 3, 1, 3, 3, 2, 3, 2, 2, 3, 2,
       3, 1, 2, 2, 1, 1, 1, 1, 1, 3, 1, 3, 1, 2, 2, 1])

In [170]:
from sklearn.linear_model import RidgeClassifier 
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss,accuracy_score,f1_score
rd_seed=1
print(Counter(target_class),"No need for class_weights...")
X_train, X_test, y_train, y_test = train_test_split(feats, target_class, test_size=0.1,random_state=rd_seed)

my_ridge_unorm_class=RidgeClassifier(normalize=True,random_state=rd_seed)
my_ridge_unorm_class.fit(X_train,y_train)
pred_unorm=my_ridge_unorm_class.predict(X_test)
print("Classification Unorm...",accuracy_score(y_test, pred_unorm),
      f1_score(y_test, pred_unorm,average='macro'))

Counter({3: 436, 2: 434, 1: 434}) No need for class_weights...
Classification Unorm... 0.419847328244 0.412433862434


In [194]:
for it,clusters_coef in enumerate(my_ridge_unorm_class.coef_):
    cluster_info=clusters_coef.argsort()
    for v in cluster_info:
        if len(phrunch_clusters[v])>500:  
            print ("Class %d most associated with clusters %d"%(it+1,v))
            break

Class 1 most associated with clusters 182
Class 2 most associated with clusters 191
Class 3 most associated with clusters 17


In [196]:
phrunch_clusters[17]

['الأمل',
 'طين',
 'تكرمت',
 'السماوات',
 'lgbtbabes',
 'دعائي',
 'منكرات',
 'أضل',
 'استطعت',
 'وأنصح',
 'السعادة',
 'est…',
 'أسألك',
 'محمداً',
 'أستخيرك',
 'علينا',
 'وهامة',
 'وإغننا',
 'وأستقدرك',
 'بشرني',
 'أعيذك',
 'الورد',
 'citroen',
 'أمي',
 'أريد',
 'نَبِيِّنَا',
 'يرحمنا',
 'والشكر',
 'السموات',
 'زد',
 'التَّامَّة',
 'صدري',
 'الذكر',
 'فوقي',
 'وأعوذ',
 'برجال',
 'والأعمال',
 'النفاق',
 'حيث',
 'فيحسن',
 'واحمينا',
 'االعظيم',
 'الكون',
 'استغفر',
 'مـن',
 'أحب',
 'الهواتف',
 'مغفرة',
 'ربَّ',
 'أتمنى',
 'دون',
 'حبه',
 'حبك',
 'مرضانا',
 'الذكية',
 'بذكرك',
 'شمالي',
 'klein',
 'جعلت',
 'تسلط',
 'والارض',
 'فاتخذت',
 'خَلَقتني',
 'قلبه',
 'قلبي',
 'وَسَلِّمْ',
 'lifethanks',
 'عيني',
 'الف',
 'الحي',
 'وشر',
 'الزمن',
 'الدرجات',
 'أقل',
 'اللهِ',
 'ذكر',
 'فزادوهم',
 'جرم',
 'اجعلنا',
 'والعطور،',
 'تتابعه',
 'تتابعي',
 'إِنِّي',
 'سنن',
 'العين',
 'اكفني',
 'أكرمنا',
 'إنِّي',
 'وعيني',
 'نسأل',
 'العظام',
 'بك',
 'حجارة',
 'القيام',
 'رضيت',
 'وانصرنا',
 'الخيانة',


In [187]:
cluster_info[::-1]

array([ 65,  49,  25, 132, 112, 128, 101, 147, 144,  81,  41, 109, 130,
        35, 159, 158,  43, 164,  69, 196, 118,  45,  29, 125,  46,  92,
       136, 133,  72, 102, 151, 134,  73, 106,  51,  22, 120, 129,  48,
       177,  28, 126,  36,  13,  77, 119, 142, 154, 123,  34, 124,  88,
        85,  21, 113,  74,  79,  82, 161,  66,  26, 114, 160,  11,  76,
        16,  33, 115, 117, 149, 194, 108,  59,  39,  32,  47,  63, 198,
        37,  57, 191,  50,  42,  98, 143, 116, 182, 185, 180, 189, 192,
         1,   2,  58, 184, 169, 181, 162, 138, 175, 172, 167, 193,   4,
       153, 166, 156,  52,  62, 155,  10, 122, 178,   3, 176, 150, 163,
       146, 188, 174, 165, 195,   6, 190, 168, 197, 173, 183,   0, 186,
        30,  83, 187, 199, 179, 152, 170,  17, 110,  24, 139,   5,  23,
       145,  54,  55, 127,  90,  87,  27,  78, 141,  20, 135,   8, 111,
       137, 157,  95, 107,  96,  94, 104,  14,  44,  40,  12, 131, 105,
         9,  97, 171, 103,  80,  60,  91,  75,  53,  18,  15,  6

In [178]:
clusters_coef[140]

-6053.9305807873934